In [1]:
%%html
<style>
.container {
        width: 100%;
        padding: 0px !important;
    }

@media (max-width:1200px) {
    #notebook[tabindex="-1"] {
        padding-top: 0px !important;
    }
    .prompt {
        min-width: 8ex;
    }
    .input_prompt {
        font-weight: bold;
        position: absolute;
        padding-right: 0px;
        padding-top: 2px;
        padding-bottom: 0px;
        padding-left: 8px;
        font-size: 10px !important;
    }
    .CodeMirror-lines {
        padding-top:12px !important;
    }
    pre {
        white-space: pre-wrap !important;
    }
}    

## Imports and set params experiments

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, re, sys
import tensorflow as tf
import tqdm
from itertools import compress
import pickle
from preprocessing import *
import glob

tf.__version__

/Users/i692122/anaconda3/envs/tf_1_8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/i692122/anaconda3/envs/tf_1_8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/i692122/anaconda3/envs/tf_1_8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/i692122/anaconda3/envs/tf_1_8/lib

'1.8.0'

##  Data preprocessing

In [2]:
path= 'data/wmt17/de-en/'
print (list(glob.iglob(os.path.join(path, '*.tok'))))
print (list(glob.iglob(os.path.join(path, '*.low'))))

['data/wmt17/de-en/europarl-v7.de-en.clean.de.tok', 'data/wmt17/de-en/commoncrawl.de-en.clean.en.tok', 'data/wmt17/de-en/newstest2013.de.tok', 'data/wmt17/de-en/news-commentary-v12.de-en.clean.en.tok', 'data/wmt17/de-en/rapid2016.de-en.clean.en.tok', 'data/wmt17/de-en/rapid2016.de-en.clean.de.tok', 'data/wmt17/de-en/commoncrawl.de-en.clean.de.tok', 'data/wmt17/de-en/newstest2013.en.tok', 'data/wmt17/de-en/news-commentary-v12.de-en.clean.de.tok', 'data/wmt17/de-en/europarl-v7.de-en.clean.en.tok']
['data/wmt17/de-en/rapid2016.de-en.clean.de.tok.low', 'data/wmt17/de-en/europarl-v7.de-en.clean.de.tok.low', 'data/wmt17/de-en/newstest2013.de.tok.low', 'data/wmt17/de-en/newstest2013.en.tok.low', 'data/wmt17/de-en/rapid2016.de-en.clean.en.tok.low', 'data/wmt17/de-en/europarl-v7.de-en.clean.en.tok.low', 'data/wmt17/de-en/train.clean.de.tok.low', 'data/wmt17/de-en/commoncrawl.de-en.clean.en.tok.low', 'data/wmt17/de-en/news-commentary-v12.de-en.clean.de.tok.low', 'data/wmt17/de-en/news-commentary

### Run preprocessing WMT17

In [3]:
# News Commentary
#-train_src data/wmt17/de-en/news-commentary-v12.de-en.clean.en.tok.low -train_tgt data/wmt17/de-en/news-commentary-v12.de-en.clean.de.tok.low -valid_src data/wmt17/de-en/newstest2013.en.tok.low -valid_tgt data/wmt17/de-en/newstest2013.de.tok.low -save_data data/news-commentary.clean.tok.low -lower -seq_length 75

# Rapid Fire
#-train_src data/wmt17/de-en/rapid*.clean.en.tok.low -train_tgt data/wmt17/de-en/rapid*.clean.de.tok.low -valid_src data/wmt17/de-en/newstest2013.en.tok.low -valid_tgt data/wmt17/de-en/newstest2013.de.tok.low -save_data data/rapid.clean.tok.low -lower -seq_length 75

# Europarl
#-train_src data/wmt17/de-en/europarl*.clean.en.tok.low -train_tgt data/wmt17/de-en/europarl*.clean.de.tok.low -valid_src data/wmt17/de-en/newstest2013.en.tok.low -valid_tgt data/wmt17/de-en/newstest2013.de.tok.low -save_data data/europarl.clean.tok.low -lower -seq_length 75

# Common Crawl
#-train_src data/wmt17/de-en/commoncrawl*.clean.en.tok.low -train_tgt data/wmt17/de-en/commoncrawl*.clean.de.tok.low -valid_src data/wmt17/de-en/newstest2013.en.tok.low -valid_tgt data/wmt17/de-en/newstest2013.de.tok.low -save_data data/commoncrawl.clean.tok.low -lower -seq_length 75

# WMT'17
#-train_src data/wmt17/de-en/train.clean.en.tok.low -train_tgt data/wmt17/de-en/train.clean.de.tok.low -valid_src data/wmt17/de-en/newstest2013.en.tok.low -valid_tgt data/wmt17/de-en/newstest2013.de.tok.low -save_data data/wmt17.clean.tok.low -lower -seq_length 75


# path IWSLT17
p = os.path.join('data','wmt17', 'de-en')

# dict of paths
dict_srcl_tgtl_path_wmt17 = {
                       'train': {
                           'src': os.path.join(p, 'train.clean.en.tok.low'), 'tgt': os.path.join(p, 'train.clean.de.tok.low')
                           },
                        'val': {
                           'src': os.path.join(p, 'newstest2013.en.tok.low'), 'tgt': os.path.join(p, 'newstest2013.de.tok.low')
                           }
                      }

# -> for preprocessing
path_save = 'data/saved_preprocess/wmt17'
# -> to load preprocessing
# path_save = 'data/saved_preprocess/wmt17_30000_50_glove_charemb.p'

if path_save.split('.')[-1] != 'p':
    # preprocess data...
    Preprocess_wmt17 = PreprocessorMT(dict_srcl_tgtl_path_wmt17,\
                     path_save=path_save,\
                     max_vocab_src=30000, max_vocab_tgt=30000, max_seq_length=50,\
                     glove_path='data/GloVe/glove.840B.300d.txt', charemb_path='data/kazuma/charNgram.txt', test=False)

    Preprocess_wmt17.process_and_save()
else:
    # or load preprocessed file
    print ('Loading the preprocessed data...')
    Preprocess_wmt17 = pickle.load(open(path_save, 'rb'))
    print ('Done loading the preprocessed data!')

#####    Processing the files    #####
Start loading training data.
Starting on src
Starting on tgt
dict_keys(['src', 'tgt'])
Done loading training data.

Start filter length training data sentences.
Loop filter max seq length...



After filtering train set: max length 'src' = 50  /  max length 'tgt' = 50
Done filter length training data sentences.

Create vocab src from training.
Done creating vocab src from training.

Create vocab tgt from training.
Done creating vocab tgt from training.

Tokenize src...



Tokenize tgt...



Loop filter max seq length...



After filtering val set: max length 'src' = 50  /  max length 'tgt' = 50
Tokenize src...


...


loading GloVe from txt file...



Creating all embeddings for src



loading Charemb kazuma from txt file...



Modify all embeddings for src with charemb



Done with vocabulary.
#####     Saving the dataset     #####
Saving the dataset...
Done saving the data
Saved at data/saved_preprocess/wmt17_30000_50_glove_charemb.p


In [5]:
for mode in ['train', 'val']:
    print ("Size of set {}: {}".format(mode, Preprocess_wmt17.get_length_set(mode)))

Size of set train: 485925
Size of set val: 345


## Modelling

In [6]:
from NMT import *
dataset_MT = dataset(Preprocess_wmt17)

default_params = {
    'dataset': None,
    'save_path': '',
    'mode': 'TRAIN',
    'nb_layers_enc': None,
    'nb_layers_dec': None,
    'embedding_dim': 300,
    'train_emb_src': True,
    'init_emb': None,
    'lstm_size_enc': 600,
    'lstm_size_dec': 600,
    'nb_layers_enc': 2,
    'nb_layers_dec': 2,
    'keep_probability': 0.8,
    'batch_size': 64,
    'epochs': 20,
    'EOS': "</s>",
    'SOS': "<s>",
    'PAD': '<pad>',
    'clip': 5,
    'lr': 0.001,
    'lr_decay': 0.9,
    'lr_decay_steps': 100,
    'print_update': 25,
    'patience': 8
}

### Start training of seq2seq model

In [7]:
reset_graph()

train_params = default_params
train_params['dataset'] = dataset_MT
train_params['lstm_size_enc'] = 600
train_params['lstm_size_dec'] = 600
train_params['nb_layers_enc'] = 2
train_params['nb_layers_dec'] = 2
train_params['embedding_dim'] = 400 # because GLoVe+charemb
train_params['init_emb'] = np.vstack(list(map(lambda i: dataset_MT.data.vocab_src.embeddings[dataset_MT.data.vocab_src.idx2lab[i]], np.arange(dataset_MT.data.vocab_src.size)))).astype(np.float32)
train_params['batch_size'] = 256
train_params['epochs'] = 1
train_params['mode'] = 'TRAIN' 
train_params['save_path'] = 'save_run/MT_L/run1'

nmt  =  NMT(train_params)
  
nmt.build_model()

model_summary()

Graph of model is built.
---------
Variables: name (type shape) [size]
---------
src_embedding:0 (float32_ref 30000x400) [12000000, bytes: 48000000]
tgt_embedding:0 (float32_ref 30000x400) [12000000, bytes: 48000000]
dynamic_seq2seq/encoder/bidirectional_rnn/fw/lstm_cell/kernel:0 (float32_ref 700x1200) [840000, bytes: 3360000]
dynamic_seq2seq/encoder/bidirectional_rnn/fw/lstm_cell/bias:0 (float32_ref 1200) [1200, bytes: 4800]
dynamic_seq2seq/encoder/bidirectional_rnn/bw/lstm_cell/kernel:0 (float32_ref 700x1200) [840000, bytes: 3360000]
dynamic_seq2seq/encoder/bidirectional_rnn/bw/lstm_cell/bias:0 (float32_ref 1200) [1200, bytes: 4800]
dynamic_seq2seq/decoder/memory_layer/kernel:0 (float32_ref 600x600) [360000, bytes: 1440000]
dynamic_seq2seq/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/kernel:0 (float32_ref 1600x2400) [3840000, bytes: 15360000]
dynamic_seq2seq/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/bias:0 (float32_ref 2400) [2400, bytes: 9600]
dynamic_se

In [8]:
nmt.train(save=True, print_nb=2, print_bleu=100)

-------------------- Epoch 0 of 1 --------------------
Iteration: 0 of 1897	train_loss: 10.3093
Overall BLEU-4 on 256 sentences: 0.0000
----------------------------------------
Example n°1:

Src sentence:  to remedy this is very expensive <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Tgt sentence:  <s> dagegen <unk> kostet viel </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Tgt predicted: <s> <s> <s> </s> </s> </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Bleu score:    0
----------------------------------------
Example n°2:

Src sente

/Users/i692122/anaconda3/envs/tf_1_8/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/i692122/anaconda3/envs/tf_1_8/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/i692122/anaconda3/envs/tf_1_8/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Con

Overall BLEU-4 on 256 sentences: 0.0000
----------------------------------------
Example n°1:

Src sentence:  donations for the new soul <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Tgt sentence:  <s> spenden für die neue seele </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Tgt predicted: <s> <s> <unk> <unk> </s> </s> </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Bleu score:    0
----------------------------------------
Example n°2:

Src sentence:  and , in a match , it 's easier . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

KeyboardInterrupt: 

### Inference

##### Inference on training set (2048 samples)

In [9]:
reset_graph()

infer_params = default_params
infer_params['dataset'] = dataset_MT
infer_params['lstm_size_enc'] = 600
infer_params['lstm_size_dec'] = 600
infer_params['nb_layers_enc'] = 2
infer_params['nb_layers_dec'] = 2
infer_params['embedding_dim'] = 400 # because GLoVe+charemb
infer_params['init_emb'] = np.vstack(list(map(lambda i: dataset_MT.data.vocab_src.embeddings[dataset_MT.data.vocab_src.idx2lab[i]],\
                                              np.arange(dataset_MT.data.vocab_src.size)))).astype(np.float32)
infer_params['batch_size'] = 2048
infer_params['mode'] = 'INFER' 
infer_params['save_path'] = 'save_run/MT_L/run1'

nmt_infer  =  NMT(infer_params)
  
nmt_infer.build_model()

model_summary()

idx = np.random.choice(dataset_MT.data.get_length_set('train'), 2048)
preds = nmt_infer.infer(dataset_MT.data.data_tok['train'], idx, restore_path = infer_params['save_path'])

Graph of model is built.
---------
Variables: name (type shape) [size]
---------
src_embedding:0 (float32_ref 30000x400) [12000000, bytes: 48000000]
tgt_embedding:0 (float32_ref 30000x400) [12000000, bytes: 48000000]
dynamic_seq2seq/encoder/bidirectional_rnn/fw/lstm_cell/kernel:0 (float32_ref 700x1200) [840000, bytes: 3360000]
dynamic_seq2seq/encoder/bidirectional_rnn/fw/lstm_cell/bias:0 (float32_ref 1200) [1200, bytes: 4800]
dynamic_seq2seq/encoder/bidirectional_rnn/bw/lstm_cell/kernel:0 (float32_ref 700x1200) [840000, bytes: 3360000]
dynamic_seq2seq/encoder/bidirectional_rnn/bw/lstm_cell/bias:0 (float32_ref 1200) [1200, bytes: 4800]
dynamic_seq2seq/decoder/memory_layer/kernel:0 (float32_ref 600x600) [360000, bytes: 1440000]
dynamic_seq2seq/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/kernel:0 (float32_ref 1600x2400) [3840000, bytes: 15360000]
dynamic_seq2seq/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/bias:0 (float32_ref 2400) [2400, bytes: 9600]
dynamic_se

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for save_run/MT_L/run1
	 [[Node: save_1/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2/tensor_names, save_1/RestoreV2/shape_and_slices)]]

Caused by op 'save_1/RestoreV2', defined at:
  File "/Users/i692122/anaconda3/envs/tf_1_8/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/i692122/anaconda3/envs/tf_1_8/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/i692122/anaconda3/envs/tf_1_8/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/i692122/anaconda3/envs/tf_1_8/lib/python3.6/site-packages/traitlets/config/application.py", line 658,

In [10]:
BLEU_set(dataset_MT, idx, preds, 'train', samples=15)

Overall BLEU-4 on 2048 sentences: 0.0184
------------------------------------------------------------
Example n°1:

Src sentence:  four boys run away up an incline .
Tgt sentence:  vier jungen rennen einen hang hinauf davon .
Tgt predicted: <s> ein ein in einem einem . . </s> </s> </s> </s>
Bleu score:    0.0
------------------------------------------------------------
Example n°2:

Src sentence:  four women interact at a conference .
Tgt sentence:  vier frauen interagieren auf einer konferenz .
Tgt predicted: <s> ein mann in einem . . </s> </s> </s> </s> </s>
Bleu score:    0.0
------------------------------------------------------------
Example n°3:

Src sentence:  a woman in a dress doing a craft .
Tgt sentence:  eine frau in einem kleid bastelt etwas .
Tgt predicted: <s> ein ein mann in einem einem . . </s> </s> </s>
Bleu score:    0.0
------------------------------------------------------------
Example n°4:

Src sentence:  men are sitting on bags on the ground .
Tgt sentence:  män

#### Inference on validation set

In [11]:
reset_graph()

infer_params = default_params
infer_params['dataset'] = dataset_MT
infer_params['lstm_size_enc'] = 600
infer_params['lstm_size_dec'] = 600
infer_params['nb_layers_enc'] = 2
infer_params['nb_layers_dec'] = 2
infer_params['embedding_dim'] = 400 # because GLoVe+charemb
infer_params['init_emb'] = np.vstack(list(map(lambda i: dataset_MT.data.vocab_src.embeddings[dataset_MT.data.vocab_src.idx2lab[i]],\
                                              np.arange(dataset_MT.data.vocab_src.size)))).astype(np.float32)
infer_params['batch_size'] = dataset_MT.data.get_length_set('val')
infer_params['mode'] = 'INFER' 
infer_params['save_path'] = 'save_run/MT_L/run1'

nmt_infer  =  NMT(infer_params)
  
nmt_infer.build_model()

model_summary()

idx = np.arange(dataset_MT.data.get_length_set('val')) # predict whole set at once
preds = nmt_infer.infer(dataset_MT.data.data_tok['val'], idx, restore_path =  infer_params['save_path'])

Graph of model is built.
---------
Variables: name (type shape) [size]
---------
src_embedding:0 (float32_ref 3023x400) [1209200, bytes: 4836800]
tgt_embedding:0 (float32_ref 3819x400) [1527600, bytes: 6110400]
dynamic_seq2seq/encoder/bidirectional_rnn/fw/lstm_cell/kernel:0 (float32_ref 700x1200) [840000, bytes: 3360000]
dynamic_seq2seq/encoder/bidirectional_rnn/fw/lstm_cell/bias:0 (float32_ref 1200) [1200, bytes: 4800]
dynamic_seq2seq/encoder/bidirectional_rnn/bw/lstm_cell/kernel:0 (float32_ref 700x1200) [840000, bytes: 3360000]
dynamic_seq2seq/encoder/bidirectional_rnn/bw/lstm_cell/bias:0 (float32_ref 1200) [1200, bytes: 4800]
dynamic_seq2seq/decoder/memory_layer/kernel:0 (float32_ref 600x600) [360000, bytes: 1440000]
dynamic_seq2seq/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/kernel:0 (float32_ref 1600x2400) [3840000, bytes: 15360000]
dynamic_seq2seq/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/bias:0 (float32_ref 2400) [2400, bytes: 9600]
dynamic_seq2seq/

In [12]:
BLEU_set(dataset_MT, idx, preds, 'val', samples=15)

Overall BLEU-4 on 148 sentences: 0.0162
------------------------------------------------------------
Example n°1:

Src sentence:  a man in a cluttered office is using the telephone
Tgt sentence:  ein mann telefoniert in einem unaufgeräumten büro
Tgt predicted: <s> ein ein mann in einem einem . . </s> </s>
Bleu score:    0.0
------------------------------------------------------------
Example n°2:

Src sentence:  a female playing a song on her violin .
Tgt sentence:  eine frau spielt ein lied auf ihrer geige .
Tgt predicted: <s> ein ein mann in einem . . . </s> </s>
Bleu score:    0.0
------------------------------------------------------------
Example n°3:

Src sentence:  a young woman is making rugs in the rain forest
Tgt sentence:  eine junge frau fertigt im regenwald teppiche an
Tgt predicted: <s> ein ein mann in einem einem . . </s> </s>
Bleu score:    0
------------------------------------------------------------
Example n°4:

Src sentence:  a cute baby is smiling at another child

#### Inference on test set

In [13]:
reset_graph()

infer_params = default_params
infer_params['dataset'] = dataset_MT
infer_params['lstm_size_enc'] = 600
infer_params['lstm_size_dec'] = 600
infer_params['nb_layers_enc'] = 2
infer_params['nb_layers_dec'] = 2
infer_params['embedding_dim'] = 400 # because GLoVe+charemb
infer_params['init_emb'] = np.vstack(list(map(lambda i: dataset_MT.data.vocab_src.embeddings[dataset_MT.data.vocab_src.idx2lab[i]],\
                                              np.arange(dataset_MT.data.vocab_src.size)))).astype(np.float32)
infer_params['batch_size'] = dataset_MT.data.get_length_set('test')
infer_params['mode'] = 'INFER' 
infer_params['save_path'] = 'save_run/MT_L/run1'

nmt_infer  =  NMT(infer_params)
  
nmt_infer.build_model()

model_summary()

Graph of model is built.
---------
Variables: name (type shape) [size]
---------
src_embedding:0 (float32_ref 3023x400) [1209200, bytes: 4836800]
tgt_embedding:0 (float32_ref 3819x400) [1527600, bytes: 6110400]
dynamic_seq2seq/encoder/bidirectional_rnn/fw/lstm_cell/kernel:0 (float32_ref 700x1200) [840000, bytes: 3360000]
dynamic_seq2seq/encoder/bidirectional_rnn/fw/lstm_cell/bias:0 (float32_ref 1200) [1200, bytes: 4800]
dynamic_seq2seq/encoder/bidirectional_rnn/bw/lstm_cell/kernel:0 (float32_ref 700x1200) [840000, bytes: 3360000]
dynamic_seq2seq/encoder/bidirectional_rnn/bw/lstm_cell/bias:0 (float32_ref 1200) [1200, bytes: 4800]
dynamic_seq2seq/decoder/memory_layer/kernel:0 (float32_ref 600x600) [360000, bytes: 1440000]
dynamic_seq2seq/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/kernel:0 (float32_ref 1600x2400) [3840000, bytes: 15360000]
dynamic_seq2seq/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/bias:0 (float32_ref 2400) [2400, bytes: 9600]
dynamic_seq2seq/

In [14]:
idx = np.arange(dataset_MT.data.get_length_set('test')) # predict whole set at once
preds = nmt_infer.infer(dataset_MT.data.data_tok['test'], idx, restore_path =  infer_params['save_path'])

Restore graph from save_run/MT_S/run1
INFO:tensorflow:Restoring parameters from save_run/MT_S/run1
Graph restored!


In [15]:
BLEU_set(dataset_MT, idx, preds, 'test', samples=15)

Overall BLEU-4 on 181 sentences: 0.0143
------------------------------------------------------------
Example n°1:

Src sentence:  people are fixing the roof of a house .
Tgt sentence:  leute reparieren das dach eines hauses .
Tgt predicted: <s> ein mann in auf einem . . </s> </s> </s>
Bleu score:    0.0
------------------------------------------------------------
Example n°2:

Src sentence:  a group of people standing in front of an igloo .
Tgt sentence:  eine gruppe von menschen steht vor einem iglu .
Tgt predicted: <s> ein ein mann auf einem einem . . . </s>
Bleu score:    0.0
------------------------------------------------------------
Example n°3:

Src sentence:  a guy works on a building .
Tgt sentence:  ein typ arbeitet an einem gebäude .
Tgt predicted: <s> ein mann in einem einem . . </s> </s> </s>
Bleu score:    0.0
------------------------------------------------------------
Example n°4:

Src sentence:  three people sit in a cave .
Tgt sentence:  drei leute sitzen in einer höh

#### Running the decoder to get CoVe Embedding

In [16]:
idx = np.arange(dataset_MT.data.get_length_set('test')) # predict whole set at once
preds = nmt_infer.infer(dataset_MT.data.data_tok['test'], idx, restore_path =  infer_params['save_path'])

Restore graph from save_run/MT_S/run1
INFO:tensorflow:Restoring parameters from save_run/MT_S/run1
Graph restored!


In [17]:
BLEU_set(dataset_MT, idx, preds, 'test', samples=2)

Overall BLEU-4 on 181 sentences: 0.0144
------------------------------------------------------------
Example n°1:

Src sentence:  people are fixing the roof of a house .
Tgt sentence:  leute reparieren das dach eines hauses .
Tgt predicted: <s> ein mann in auf einem . . </s> </s> </s>
Bleu score:    0.0
------------------------------------------------------------
Example n°2:

Src sentence:  a group of people standing in front of an igloo .
Tgt sentence:  eine gruppe von menschen steht vor einem iglu .
Tgt predicted: <s> ein ein mann auf einem einem . . . </s>
Bleu score:    0.0
------------------------------------------------------------


In [18]:
emb = nmt_infer.CoVe(dataset_MT.data.data_tok['test'], idx, restore_path=infer_params['save_path'], with_emb=True) # with_emb to add or not original embeddings (GloVe+char emb)

Restore graph from save_run/MT_S/run1
INFO:tensorflow:Restoring parameters from save_run/MT_S/run1
Graph restored!


Verifying the number of tokens is matching the non-zero embedding outputed by the model.
Sentence 0 has 9 tokens while sentence 1 has 11 tokens.

In [26]:
emb.shape

(181, 50, 1000)

In [24]:
((emb[0]!=0).sum(axis=1)==1000).sum()

9

In [25]:
((emb[1]!=0).sum(axis=1)==1000).sum()

11